In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs

In [2]:
data = pd.read_csv('dataset_after1.csv',index_col=0)
data.head()

,Molecule ChEMBL ID,Smiles,Standard Relation,Standard Value,Standard Units
0,CHEMBL2263120,CC1(C)CC[C@]2(C(=O)OCCCCCCCCCCC(=O)O)CC[C@]3(C...,'=',0.7200,nM
1,CHEMBL2263121,CC1(C)CC[C@]2(C(=O)NC(Cc3ccccc3)C(=O)O)CC[C@]3...,'=',3.3100,nM
2,CHEMBL168,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,'=',5.0501,nM
3,CHEMBL508831,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,'=',0.5400,nM
4,CHEMBL506556,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,'=',0.6200,nM


In [3]:
data.columns

Index(['Molecule ChEMBL ID', 'Smiles', 'Standard Relation', 'Standard Value',
       'Standard Units'],
      dtype='object')

In [4]:
list_smiles = data["Smiles"].values.tolist()
list_smiles = [Chem.MolFromSmiles(x) for x in list_smiles[:5]]
list_smiles

In [5]:
fps = [Chem.RDKFingerprint(x) for x in list_smiles]
fps

In [6]:
fps_list = []
for i in range(len(fps)):
    for j in range(len(fps)):
        if (j > i):
            tmp = DataStructs.FingerprintSimilarity(fps[i],fps[j])
            fps_list.append(tmp)

In [7]:
similarity = np.average(fps_list)
similarity

0.6369825435809657

In [8]:
simLs = [
    [1,1,1,1,1],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
]
n = len(simLs)

n = 3

simLs = np.zeros([n,n])
print(n)

divLs = np.ones([n,n])
for i in range(n) :
    for j in range(n) :
        dx = 1-simLs[i][j]
        fx = np.exp(-0.5*dx)
        divLs[i][j] = fx
print(divLs)

3
[[0.60653066 0.60653066 0.60653066]
 [0.60653066 0.60653066 0.60653066]
 [0.60653066 0.60653066 0.60653066]]


In [9]:
def diversity(n):
    simLs = np.zeros([n,n])
    matDiv = [np.ones([n,n]),np.zeros([n,n])]

    matSumDiv = []
    for i in range(len(matDiv)) :
        simLs = matDiv[i]
        divLs = np.ones([n,n])
        for i in range(n) :
            for  j in range(n) :
                dx = 1-simLs[i][j]
                fx = np.exp(-0.5*dx)
                divLs[i][j] = fx
        #print(divLs)


        #handle determinan = 0

        handleDet = 0.0000000001
        countDet = 1
        sameCorr = []
        for i in range(n) :
            for j in range(n) :
                if i > j :
                    if divLs[i][j] in sameCorr :
                        if divLs[i][j] == 0.0 :
                            a = divLs[i][j] + (countDet*handleDet)
                        else :
                            a = divLs[i][j] - (countDet*handleDet)
                        #print(a)
                        #print(divLs[i][j],a)
                        divLs[i][j] = a
                        divLs[j][i] = a
                        countDet += 1
                    else :
                        sameCorr.append(divLs[i][j])
        #print(divLs)



        #print(divLs)
        #print(np.linalg.det(divLs))
        #print(divLs)
        e = np.ones(n)
        eT = np.transpose(e)
        divLsInv = np.linalg.inv(divLs)
        #print(divLsInv)
        sumDiverse = e.dot(divLsInv).dot(eT)
        matSumDiv.append(sumDiverse)

    return matSumDiv
diversity(10)[1]

1.648718386888504

In [10]:
import pandas as pd
nList = [5,10,25,50,100]

minLs = []
maxLs = []
dicta = {x : {} for x in (nList)}
#print(dicta)
for i in nList :
    dicta[i]["min"] = diversity(i)[0]
    dicta[i]["max"] = diversity(i)[1]
print(dicta)  

dfDivers = pd.DataFrame(dicta)
dfDivers

{5: {'min': 0.9999990463256836, 'max': 1.6487185955047607}, 10: {'min': 1.000000586733222, 'max': 1.648718386888504}, 25: {'min': 0.9999998887942638, 'max': 1.6487182869605022}, 50: {'min': 1.000000504811375, 'max': 1.6487211864717466}, 100: {'min': 0.9999996809032357, 'max': 1.6487192886860622}}


,5,10,25,50,100
min,0.999999,1.000001,1.000000,1.000001,1.000000
max,1.648719,1.648718,1.648718,1.648721,1.648719
